In [2]:
# Develop features for demo, etc. 
# But, first step is to create DF from Recommendation
import collections
import compress_pickle
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")
RESULT_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/result_files'

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [107]:
def recommendSpecies():
    # method to directly interact with user; 
    # simple display purpose
    # TODO: Once it's displayed,
    # save it in the recom class;
    pass
def recommendReaction():
    # method to directly interact with user; 
    # simple display purpose
    pass

def getDataFrameFromRecommendation(rec, show_url=False):
    """
    Get a pandas dataframe from 
    a single recommendation.

    Parameters
    ----------
    rec: cn.Recommendation

    Returns
    -------
    :str
    """
    cands = [val[0] for val in rec.candidates]
    match_scores = [val[1] for val in rec.candidates]
    
    labels = rec.labels
    # index starts from 1;
    df = pd.DataFrame({'annotation':cands,
                       'match_score':match_scores,
                       'label':labels},
                       index=[1+val for val in list(range(len(cands)))])
    df.index.name = 'ID: %s' % rec.id
    if show_url:
      urls = rec.urls
      df['url'] = urls
    return df

def getMarkdownFromRecommendation(rec, show_url=False):
    """
    Get a markdown from 
    a dataframe.

    Parameters
    ----------
    rec: cn.Recommendation

    Returns
    -------
    :str
    """
    df = getDataFrameFromRecommendation(rec, show_url)
    # In markdown, title is shown separately,
    # so index name with element ID is removed; 
    df.index.name=None
    df_str = df.to_markdown(tablefmt="grid", floatfmt=".03f", index=True)
    # Centering and adding the title 
    len_first_line = len(df_str.split('\n')[0])
    title_line = "%s (credibility score: %.03f)" % (rec.id,  rec.credibility)
    title_line = title_line.center(len_first_line)
    df_str = title_line + '\n' + df_str
    return df_str

In [108]:
E_COLI_PATH = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS/AMAS/../tests/e_coli_core.xml'
BIOMD_190_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000190.xml')

In [109]:
recom = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)
one_rec = recom.getSpeciesAnnotation(pred_id='SAM')

In [110]:
one_rec

Recommendation(id='SAM', credibility=0.975, candidates=[('CHEBI:59789', 1.0), ('CHEBI:15414', 1.0)], urls=['https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A59789', 'https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI%3A15414'], labels=['S-adenosyl-L-methionine zwitterion', 'S-adenosyl-L-methionine'])

In [134]:
RESULT_RECOM = cn.Recommendation('R_PFK', 0.817,
                                 [('RHEA:12420', 0.6), ('RHEA:13377', 0.6)],
                                 ['https://www.rhea-db.org/rhea/12420', 'https://www.rhea-db.org/rhea/13377'],
                                 ['tagatose-6-phosphate kinase activity', 'phosphoglucokinase activity'])

In [140]:
df = getDataFrameFromRecommendation(RESULT_RECOM, show_url=True)
df

,annotation,match_score,label,url
ID: R_PFK,,,,
1,RHEA:12420,0.6,tagatose-6-phosphate kinase activity,https://www.rhea-db.org/rhea/12420
2,RHEA:13377,0.6,phosphoglucokinase activity,https://www.rhea-db.org/rhea/13377


In [145]:
1 in df.index

True

In [137]:
getMarkdownFromRecommendation(RESULT_RECOM, show_url=False)

'                      R_PFK (credibility score: 0.817)                      \n+----+--------------+---------------+--------------------------------------+\n|    | annotation   |   match_score | label                                |\n+====+==============+===============+======================================+\n|  1 | RHEA:12420   |         0.600 | tagatose-6-phosphate kinase activity |\n+----+--------------+---------------+--------------------------------------+\n|  2 | RHEA:13377   |         0.600 | phosphoglucokinase activity          |\n+----+--------------+---------------+--------------------------------------+'

In [131]:
def selectAnnotation(choice=None):
  """
  Based on the previous recommendation,
  determine the annotations to store.
  If None given, select all.
  
  Parameters
  ----------
  choice: int/list-int/None
  """
  if isinstance(choice, int):
    chosen = [choice]
  elif isinstance(choice, list):
    chosen = choice
  elif choice is None:
    return df
  # Update previous interactions;
  return df.loc[chosen, :]

In [132]:
df.index.name.split(' ')[-1]

'SAM'

In [133]:
selectAnnotation()

,annotation,match_score,label
ID: SAM,,,
1,CHEBI:59789,1.0,S-adenosyl-L-methionine zwitterion
2,CHEBI:15414,1.0,S-adenosyl-L-methionine


In [117]:
df.loc[[1], ['annotation', 'match_score']]

,annotation,match_score
ID: SAM,,
1,CHEBI:59789,1.0


In [62]:
one_dict = {'SAM': df}
one_dict['SAM']

,annotation,match_score,url,label
ID: SAM,,,,
1,CHEBI:59789,1.0,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...,S-adenosyl-L-methionine zwitterion
2,CHEBI:15414,1.0,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...,S-adenosyl-L-methionine
